# Zusammenfassungen und Erzählungen


## VL1: Raw Rates, methods (YoB, YoD, MortCoeff, Exposure)
Beginnen wir eine kurze Zusammenfassung der Vorlesung.
In der ersten Vorlesung wurde erstmals das Konzept der Sterberaten eingeführt als ANzahl der Tode durch Anzahl der Lebenden. Für die Berechnung der Sterberaten wurden verschiedene Konzepte und Schätzer eingeführt. 
* Die erste davon ist die Geburtsjahr-Methode, welche die Toten aus demselben Geburtsjahr nimmt. Diese können entweder im aktuellen oder aber im nächsten Jahr sterben, wenn sie noch vor ihrem Geburtstag sterben, der sie in die nächste Altersklasse wirft. 
* Die zweite ist die Sterbejahrmethode: Hier kommt es darauf an, wie alt man im effektiven Sterbejahr ist, egal ob man im einen oder anderen Jahr geboren wurde. 
* Schließlich gibt es noch einen Schätzer mit dem "Mortality coefficient", der ebenfalls auf das Sterbejahr setzt, aber ein wenig eine andere Methodik benutzt und damit erstmalig auch etwas über Migrationseffekte nachdenkt.
* Die Exposure-Methode entspricht im Wesentlichen einer Risk-pro-Rata Idee und flexibilisiert den Nenner etwas hin zu einer realistischeren Abschätzung der Anzahl der lebendigen. Durchaus gescheit - nur könnte er für gewisse Fälle zu einer unrealistisch hohen Sterberate führen. Dieses Problem beseitigen wir, indem wir die Exposure der STerbenden (und, vermutlich, der emigrierenden) jeweils auf eins setzen. 

## VL2: Smoothing: Mechanical/Whittaker-Henderson
Auch die Idee der zweiten Vorlesung ist einigermaßen rasch erzählt. Im Wesentlichen geht es nun darum, die beobachteten Sterberaten etwas glatter zu gestalten, vermutlich v.a. im Hinblick auf die späteren Ideen von Forecasting. Besonders wild fluktuierende Sterberaten in x oder t scheinen auch wenig sinnvoll. Dafür gibt es verschiedene Methoden, von denen wir insbesondere das Smoothing mit Splines für wichtig genommen haben. 
* Mechanical Smoothing: Das ist die trivialste Methode. In einer Umgebung von i-r bis i+s wird gewichtet über alle beobachteten Sterberaten $\tilde q$ gemittelt. 
* Eine doppelte Mittlung mit jeweils einer Umgebung von $\pm 2$ erfolgt bei der Methode von Finliaison Wittstein mit Gewicht 1/5. Durch die doppelte Mittlung erweitert sich der endgültige Bereich zwar, wird jedoch an den Rändern weniger stark berücksichtigt als in der "Mitte". 
* Whittaker-Henderson hat als Idee, gleichzeitig auf Abweichung (Fitting) als auch auf Glattheit (Smoothing) hin zu optimieren: Dafür führen wir ein Maß der Glattheit (Smoothness) ein, das wir mithilfe des Konzepts der diskreten Ableitung  definieren: $\Delta ^k x_i = \sum_{j=0}^k (-1)^{k-j} \binom{k}{j} x_{i+j} $, $S^d (q) = \sum_{k = 0}^{w-d} \Delta^k q_k$. Eigentlich ist das eher ein Maß für die Un-Smoothness, da wir sagen, dass etwas "glatter" ist wenn $S^d$ kleiner ist. Aber das soll uns hier nicht weiter stören. Nun brauchen wir also noch die Abweichung, die trivial definiert ist: Für $q$ die geglättete und $\tilde q$ die beobachtete Sterberate definieren wir als Abweichung $\sum_{k=0}^w w_k (q_k - \tilde q_k)^2$. Nun gilt es also, das ganze zu minimieren, indem wir die Summe der beiden minimieren und dabei noch einen Grad $g$ definieren, mit dem wir die Smoothness betrachten. In der Praxis ist d meist zwischen 2 und 3 herum gewählt. 
Wir lösen das mathematische Minimierungsproblem durch eine Transformation des Problems in eine Matrixgleichung und optimieren analytisch nach einem $q$, das wir ja finden wollen. So finden wir ein optimales $q$. Das ganze ist hier auch implementiert.

## VL3: Cubic natural splines
Die Haupt-Smoothing-Geschichte war diejenige mit den kubischen natürlichen Splines. Kubisch, weil wir mit stückweisen Polynomfunktionen höchstens dritten Grades die Partition auf einem Intervall interpolieren, sodass die Gesamtfunktion 2-glatt ist. Im Wesentlichen ist das auch schon die ganze GEschichte. Der Spline wird mit $S$ bezeichnet, die einzelnen Polynome auf der Partition mit $S_k$. Es gibt einen Operator ohne Namen, $\mathcal I f = \sum_{k=0}^w (\tilde q_k - f(x_k)) + \lambda \int_{x_0}^{x_k} f'' (x) ^2\textup{d} x$. Interessanterweise passiert die Geschichte hier nun ohne ein Gewicht bei der Deviation. Wie auch immer definieren wir eine Basis $N_j$ für die Splines und setzen den Koeffizientenvektor $b$, sd. $S = Nb$. Dann können wir abermals mit einer Umformung in eine Matrixgleichung die Splines einfach berechnen, sodass das gesamte Splineproblem minimiert wird. Am Ende berechnen wir eine solche Matrix, sodass wir für die geglätteten $q$ schreiben können $q = S_\lambda \tilde q$.
Eigentlich schon in **VL4**, aber thematisch noch hier  zugehörig ist die optimale Wahl eines Lambda. Dazu bemühen wir eine Cross-Validation und wollen natürlich den Loss dazu minimieren. Wir berechnen also $CV(\lambda) = \sum_{k=0}^w (\tilde q_k - S_\lambda^(k)(x_k))^2$, wobei der Spline hier ohne Nutzung der k-ten Beobachtung berechnet wird. Man kann nun auch zeigen, dass wir das umformen können nach $\sum_{k=0}^w \frac{(\tilde q_k - S_\lambda(x_k))^2}{(1 - S_\lambda (k,k))}.$ Das vereinfacht die Berechnung für beliebige Lambda und natürlich können wir _dann_ auch nach $\lambda$ hin minimieren.

## VL4: (CV/LOOCV), Bias/Variance-Tradeoff
In dieser Vorlesung wurden die Überlegungen zu den kubischen Splines beendet und wir fahren mit Überlegungen zu verschiedenen Fehler-Typen zusammen. Hierbei betrachten wir den mse (mean squared error), den pse (predictive squared Error) und den bias mit ihren Definitionen und führen diese ineinander über: $\textup{pse} = \textup{mse} + \sigma^2$, $b_\lambda = \textup{mse} - \textup{Var}$, wobei $\sigma$ der irreducible error der Beobachtungen ist und die Varianz sich auf den Schätzer bezieht. Irritierenderweise war's das dann auch schon in dieser Vorlesung. Inhaltlich verwandt ist dann noch die Erstellung des Global measures for performance, was aber einfach nur aus einem Mittelwert aller mse bzw. pse besteht, nicht sehr spannend. Bei der Berechnung von Varianz und Bias für Splines soll man einfach stets daran denken, wie für Splines das geglättete $q$ ausgedrückt werden muss, der Rest folgt.

## VL5 Statistical Tests
Es geht weiter im Text mit der Frage, wie wir die geglätteten $q$ nun mithilfe statistischer Tests auf ihre Güte hin überprüfen können. Dafür werden drei Tests eingeführt: 
1. Sign-Test: Wir definieren eine simple Test-Statistik, die summiert ob die Abweichung zwischen Beobachter und Schätzer >0 ist: $T = \sum_{k} 1_{A_k - E_k > 0}$. Diese Teststatistik sollte mit Bin(n+1, 1/2) gehen. Wir bestimmen dann einen kritischen Wert $n_\alpha$ sd $\mathbb P[T < n_\alpha] + \mathbb P[T > n+1-n_\alpha ] \leq \alpha $. Sollte unsere berechnete Teststatistik nun also kleiner als dieses $n_\alpha$ (oder größer...) sein, verwerfen wir die Hypothese, dass wir einen sinnvollen glattheits-Schätzer definiert haben und eine systematische Abweichung finden. Der Schätzer ist sehr simpel, entdeckt aber eigentlich v.a. systematische Tendenzen, aber nicht lange Abweichungen (sofern sie wieder ausgeglichen werden) noch wenige, dafür massive Abweichungen.
2. Iteration-Test. Ähnlich, mit leicht anderer Teststatistik: $T = \sum_{k} 1_{\textup{sgn}(A_k - E_k) \neq  \textup{sgn}(A_{k-1} - E_{k-1})}$. Auch hier definieren wir wieder einen kritischen Wert $n_\alpha$ für eine Verteilung Bin(n, 1/2). Auch diesem Test ist das Ausmaß der Abweichungen wurscht.
3. $\chi^2$-Test, der bekannteste. Das ist jetzt etwas anders. Zunächst einmal gehen wir davon aus, dass die tatsächlichen Sterbezahlen $D_k$ binomial mit (l_k, q_k) verteilt sind.  Das heißt: Wir haben Erwartungswert $q_k l_k$ und Varianz $q_k p_k l_k$. Wenn wir nun also  $D_k - q_k l_k$ betrachten und durch die Standardabweichung teilen, $\frac{D_k - q_k l_k}{\sqrt{q_k p_k l_k}}$ geht das ganze gemäß CLT wie eine Normalverteilung. Nehmen wir davon dann das Quadrat und summieren, landen wir bei einer Chi-Quadrat Verteilung mit n+1 Freiheitsgraden. Mit Quantil $1-\alpha$ können wir also die Null-Hypothese ablehnen, wenn die Teststatistik $T = \sum_k \frac{(D_k - q_k l_k)^2}{{q_k p_k l_k}} > \chi^2_{n+1, 1-\alpha}$. Allerdings hat auch dieser Test Nachteile.

## VL6 Interpolation: Linearity, balducci, CFOM; (Trends: Nolfi)
Was genau hier überall "Interpolation" sein soll, erschließt sich mir nicht ganz, weil unterjähriges Zeug mir nicht besonders interessant erscheint. Aber gut. Es gibt hier im Wesentlichen die drei genannten Ansätze, für die etwas Terminologie notwendig ist. Die Hazard Rate entspricht im wesentlichen der Gefahr des instantanen Todes daher auch der Name "Force of Mortality". In allen Fällen ist das Ziel, die Survivla Function, i.e. $_t p_x$ herzuleiten. Für die Linearity Assumption können wir außerdem zeigen, dass im linearen Fall folgt, dass $K$ und $U$ unabhängig sind und $U$ uniform. 

## VL7 Trends in Mortality: Nolfi, Cohort, Combined // 1st-Order Tables
Here we want to look at different models to predict the future mortality rates given the measured (and smoothed) mortality rates discussed so far. We start with the first three, very simple approaches
1. Nolfi Model: $q_{x,t} = q_{t_m} e^{-F_x ( t-t_m)}$. Jetzt ist natürlich mal wieder die Frage, wie wir das $F_x$ abschätzen. Hierfür definieren wir eine Art Loss und schreiben dann $L = (\log q_{x,t} - (\alpha -F_x))^2$ und minimieren $\sum_x \sum_t L^2$. Das gibt uns $\alpha$ und $F$, wofür auch immer wir das alpha brauchen sollen. Soweit nicht klar. Ich nehme an, ähnlich geht es dann mit...
2. Cohort Model: $q_{x,t+1} = q_{x,t} e^{-G_g}, g = t-x+1$ oder dem 
3. Combined model:   $q_{x,t+1} = q_{x,t} e^{-\alpha F_x  - (1-\alpha) G_g}, g = t-x+1$

Bei den 1st-Order Tables geht es darum, ein relative Loading durchzuführen, um eine gewisse Risikopufferung zu haben. Das ist im Wesentlichen also die Beantwortung der Frage: Für gegebene Sterbezahlen $D_x$, wie können wir ein Loading $1+S_\alpha$ so gestalten, dass die Wahrscheinlichkeit, dass die Sterbezahlen größer/kleiner (je nachdem ob Todesfall oder Rente) sind als die Erwartung derselben? Dafür definieren wir genau diese W'keit (für einen Fall jetzt mal): $\mathbb P [\sum_x D_x \leq \sum l_x q_x * (1+S_\alpha)] \leq 1 - \alpha$. Wir gehen dann abermals davon aus, dass $D_x \sim \textup{Bin} (l_x, q_x)$ und leiten uns genau so her, wie S_alpha gehen müsste. Dasselbe kann man für _random payouts_ machen, wenn diese irgendwie Sinn ergeben, dafür benutzt man dann die Eigenschaften einer compound distribution, der REst bleibt gleich und auch das Ergebnis zeigt, dass abermals das Safety load mit der inversen Quadratwurzel der Population size geht. Der Sinn dahinter ist: Eigentlich interessieren uns weniger die Anzahl der Tode als die Summe der Payouts, da es da ja noch weitere Korrelationen zum Gehalt/Lebensstandard/... geben kann. 

## VL8 Lee-Carter Modell
Das ist nun eben ein anderes Modell, um die zeitliche^ Entwicklung vorherzusagen. Wir schreiben hierfür $\log q_{x,t} = a_x + b_x k_t + \varepsilon_{x,t}$. Wir definieren auch hierfür erneut einen Loss, nämlich $L = \sum_{x,t} (\log q_{x,t} - (a_x + b_x k_t))^2$. Um $a_x$ nun zu ermitteln, leiten wir nach $a_x$ ab und setzen auf null, d.h. wir minimieren den Loss nach $a_x$. Für die Ermittlung der beiden anderen Parameter $b$ und $k$ nutzen wir die SVD-Methode, nachdem wir definiert haben $ f_{x,t} = \log q_{x,t} - a_x \approx b_x k_t$ und F = $f_{x,t}$_{x,t}.$ In Matrixschreibweise haben wir also dann einen neuen "Loss" $\left\| F - b k^T \right\|$, den es zu minimieren gilt. Wir wissen, dass dieser Loss mit der SVD-Methode minimiert werden kann. Das können wir auch beweisen, auch wenn's mühsam ist. 

## VL9 Forecasting und AFM
Hier ist die Aufgabe, $k_t$ für künftige $t$ vorherzusagen. Im Wesentlichen wollen wir $k_t$ als random walk beschreiben, sd $k_{t+1} = k_t + \Delta k + z_t$. Dabei ist ein Schätzer $\Delta k = \frac{1}{T} (k_T - k_0)$. Und ein Schätzer für die Varianz von $z_t \sim \mathcal{N}(0, \sigma^2)$ ist $\frac{1}{T-1}\sum_t (k_{t+1} - {k_t} - \Delta k)^2$. Wir können dann auch angeben, was das Konfidenzintervall ist, das haben wir aber nicht hergeleitet. Nun wurde auch die sogenannte indefinitie divergence hergeleitet. Für zwei verschiedene Modelle ist diese definiert als $\lim_{t \rightarrow \infty} \mathbb E \left[ \log  \frac{ q_{x,t}^1}{ q_{x,t}^2} \right] = \infty$. Nehmen wir nun eine leicht andere Notation für den random walk: $k_{t+1} = k_t + \mu_t + ...$. Wir bezeichnen als $\mu$ damit den Drift. Wir haben keine indefinite divergence für bestimmte BEdingungen, die wir kennen. 

Als Augmented Model bezeichnen wir ein Modell, das neu auch verschiedene Populationen berücksichtigt, aber auch einen populationsübergreifenden Teil hat: $\log q^i_{x,t} = a_x^i + b_x k_t + b_x^i k_t^i + \varepsilon_{x,t}^i$. Die Strategie um das herauszufinden: a wie gehabt, dann die übergreifenden Faktoren mit einem gemittelten F über alle $F_i = \log q_i - a_x^i$, dann die separaten b, k mit $G_i = \log q_i - a_x^i - b k$

Am einfachsten lässt sich indefinite divergence vermeiden, wenn $k_t+1 = \alpha k_t + \beta$ mit Bedingungen $|\alpha| < 1$

## VL10 FFNN
in dieser Vorlesung ging es darum, wie man Feed-Forward Neurale Netze zutzen kann, um Mortality Rates vorherzusagen. Wichtig dafür ist wohl, dass man ein Neuron und ein Network Layer definieren und aufschreiben (und unterscheiden, hihi) kann, dass man es aufmalen kann. Optimiert wird mit Gradient Descent, vielleicht ist das auch nochmals einen Blick wert. Insbesondere legen wir Wert auf die Formel $\theta_{t+1} = \theta_t - \eta_t \nabla_\theta L(\mathcal{D}, \theta)|_{\theta = \theta_t}$. Es gibt ebenso die Stochastic Gradient Descent Methode, zB ADAM, wo die Idee ist dass man in Batches durch ein Set geht und in einer Epoche durch alle Daten. Das kann die Dinge schneller gestalten und die Konvergenz erhöhen. Oszillationen sind ein häufiges Problem, weshalb man einen Impulsbasierten Ansatz wählt.

Ein wichtiges Konzept ist auch die Backpropagation. Wir kennen das KOnzept, aber probieren wir es mal noch an einem Beispiel. Es ist _auch_ wichtig, ans Pre-Processing zu denken: Kontinuierliche Variablen über min-max scaler, diskrete variablen über one-hot-encoding oder imt embedded layers. Auch schön. Es gibt Drop-Out Layers und Skip-connections sowie normalisation layers. Und NN Ensembles. 

## VL11 CNN
Convolutional NNs sind eine schöne Sache. Im Gegensatz zu FFNNs müssen wir hier gar keine Features vordefinieren, das Netzwerk erkennt Strukturen selbst. Außerdem ist es effizienter und schneller. Die Building Blocks sind ein Activation Layer mit nichtlinearer Activation Funktion, ein Pooling und ein Convolution Layer sowie ein Fully Connected Layer am Ende. Wenn wir kurz über den Input nachdenken, stellen wir fest, dass wir Kanäle haben und eben räumliche Dimensionen, d.h. $z \in \mathbb{R}^{q_1 \times ... \times q_K}$. Im Fall einer Zeitreihe haben wir natürlich nur eine Dimension außer den Kanälen. 

Ein Filter/Kernel ist ein kleines FEnsterchen mit ebenso vielen räumlichen Dimensionen wie der Input. Dabei ist nur die ANzahl der Kanäle gleich und der Rest kleiner-gleich in der AUsdehnung. Das führt dann zu einem Output der Größe $q_m^{(k)} := q_{m-1}^{(k)} - f_m^{k} + 1$
Dann gibt es noch eine CNN Convolution (eher cross-correlation), je nach Aktivierungsfunktion, kommt da dann etwas anderes dabei heraus. Die Formel ist gar nicht so kompliziert wie gedacht. Es gibt noch die Begriffe **stride**, **padding** und dann kommt man bei einer Input size von $((i + 2p - k)/s) +1 $. Pooling Layers haben den Benefit, dass die Größe der ganzen Geschichte reduziert wird, was Kosten reduziert und die Konvergenz beschleunigt. Außerdem wird die Geschichte damit robuster und wichtige Sachen werden gehighlighted, overfitting wird auch weniger schlimm. Max-Pooling und Average-Pooling sind 'a thing'. Die Output size ist wieder gleich, nur ohne p und mit unterer Gaußklammer. 

CNNs sind besser für die spatial Zusammenhänge, kostengünstiger, effizienter - und haben weniger Parameter. Also alles in allem ein Gewinn. Receptive field ist ein wichtiges Stichwort. Trainiert wird mit GD, von der architektur geht's Input -> Convolution+Activation -> Pooling -> Fully Connect. In der Vorhersage von Sterberaten benutzen wir gerne eindimensionale CNNs (oder auch 2dim, aber das haben wir nicht genauer gemacht, bspw. plus Alter als Dimension). Idee: Wir scalen log q mit min-max (-> y) und benutzen ein CNN um aus der Geschichte features $z_f(Q)$ zu extrahieren, $Q = (q_{x,t})_{x,t}$. Das kombinieren wir mit Embeddings für das Geschlecht und das Land $z_G(g), z_C(c)$. Und dann gilt wieder $(z_f(Q),z_G(g), z_C(c)) \mapsto y$

## VL12 Survival Analysis